[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Helix-n-Sheet/convertToNonStandardRes/blob/main/convertToNonStandardRes.ipynb)

# Colab Prep

In [ ]:
!git clone https://github.com/Helix-n-Sheet/convertToNonStandardRes

In [ ]:
# Install py3Dmol (https://3dmol.csb.pitt.edu/)
# for visualization purposes:
try:
    import py3Dmol
except:
    !pip install py3Dmol
    import py3Dmol

In [ ]:
# Install MDAnalysis
# for loading and parsing the .pdb file
try:
    import MDAnalysis
except:
    !pip install MDAnalysis 
    import MDAnalysis

In [ ]:
# Install latest version of AmberTools
# for adding missing atoms
tleap_output = !tleap -h
if 'tleap: command not found' in tleap_output:
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda config --add channels conda-forge
    !conda install ambertools
else:
    print('Already have tleap installed.')

# Preamble

In [ ]:
# Import standard python packages
# for data provenance:
import datetime
from pathlib import Path

# for running the tleap command:
import subprocess

# User-defined parameters:

In [ ]:
nsaa_residue_number = 56   # one-indexed integer value for the residue of interest, as seen in the .cif/.pdb file
nsaa_residue_name   = 'HYP' # three letter resname, should be recognizable by the molecular mechanics forcefield
starting_structure = Path('/content/convertToNonStandardRes/test/test_protein.pdb')
output_dir = starting_structure.parent

# Load Test System

In [ ]:
# load AlphaFold inferred model into a MDAnalysis Universe
u = MDAnalysis.Universe(starting_structure)
# select all atoms as an AtomGroup
sel = u.select_atoms('all')

In [ ]:
# let's visualize the wild-type structure
before = py3Dmol.view()
with open(starting_structure,'r') as struct_file:
    before_model = struct_file.read()
before.addModel(before_model,'pdb')
# draw the newcartoon representation, color with a light blue
before.setStyle({'cartoon': {'color': '#ADD8E6'}})
# draw the to-be-modified residue with licorice/stick representation 
# but we also need to add the cartoon rep too :( 
before.setStyle({'resi':[nsaa_residue_number]}, 
                {'stick':{'colorscheme':'cyanCarbon'},
                 'cartoon':{'color': 'red'}})

before.show()

In [ ]:
# Adding data provenance information to the preamble of the PDB file
remark_list = ['Structure file written using MDAnalysis.',
               'Script to produce this structure is https://github.com/Helix-n-Sheet/convertToNonStandardRes/',
               f'Date: {datetime.datetime.now()}', 
               f'Removing sidechain atoms for residue {nsaa_residue_number}.']
u.trajectory.remarks = remark_list

### checkout the wild-type residue

In [ ]:
# because python, iterators' first index is 0 so subtract 1 from the residue number in the .pdb file
nsaa_residue_index = nsaa_residue_number - 1
original_residue_name = sel.residues[nsaa_residue_index].resname

nsaa = sel.residues[nsaa_residue_index]
print(nsaa)
for atom in nsaa.atoms:
    print(atom)

## Create the temporary structure file

In [ ]:
# create a 2nd atom selection group, where we grab all atoms except side chain atoms of residue of interest
modified_sel = sel.select_atoms(f'not (resid {nsaa_residue_number} and not backbone)')
# for the residue of interest, change the residue name to the desired nsaa three character string
modified_sel.residues[nsaa_residue_index].resname = nsaa_residue_name
# write out the temporary structure file
modified_sel.write(output_dir / 'temp.pdb')

In [ ]:
# let's visualize the temporary structure
temp = py3Dmol.view()
with open(output_dir / 'temp.pdb','r') as struct_file:
    temp_model = struct_file.read()
temp.addModel(temp_model,'pdb')
# draw the newcartoon representation, color with a light blue
temp.setStyle({'cartoon': {'color': '#ADD8E6'}})
# draw the to-be-modified residue with licorice/stick representation 
temp.setStyle({'resi':[nsaa_residue_number]}, 
                {'stick':{'colorscheme':'cyanCarbon'},
                 'cartoon':{'color': 'red'}})

temp.show()

In [ ]:
# checking our work; load the new temp.pdb file
u = MDAnalysis.Universe(output_dir / 'temp.pdb')
# create an atom selection group
sel = u.select_atoms(f'resid {nsaa_residue_number}')
print(sel.residues[0])
for atom in sel.atoms:
    print(atom)

# Load the temp pdb into tleap and write out the modified model

In [ ]:
with open(output_dir / 'tleap.in','w') as file:
    file.write(f'source leaprc.protein.ff19SB\nprot = loadpdb temp.pdb\nsavepdb prot modified.pdb\n\nquit')
subprocess.run(f'tleap -s -f tleap.in',shell=True,cwd=output_dir)

In [ ]:
u = MDAnalysis.Universe(output_dir /'modified.pdb')
sel = u.select_atoms('all')

# Adding data provenance information to the preamble of the PDB file
remark_list = ['Structure file written using tLeap and MDAnalysis.',
               'Script to produce this structure is https://github.com/Helix-n-Sheet/convertToNonStandardRes/',
               f'Date: {datetime.datetime.now()}', 
               f'Converted residue {nsaa_residue_number} from {original_residue_name} to {nsaa_residue_name}']

u.trajectory.remarks = remark_list
sel.write(output_dir / 'modified.pdb')

In [ ]:
# let's visualize the modified structure
after = py3Dmol.view()
with open(output_dir / 'modified.pdb','r') as struct_file:
    after_model = struct_file.read()
after.addModel(after_model,'pdb')
# draw the newcartoon representation, color with a light blue
after.setStyle({'cartoon': {'color': '#ADD8E6'}})
# draw the to-be-modified residue with licorice/stick representation 
# but we also need to add the cartoon rep too :( 
after.setStyle({'resi':[nsaa_residue_number]}, 
                {'stick':{'colorscheme':'cyanCarbon'},
                 'cartoon':{'color': 'red'}})

after.show()

# Visualize all three structures

In [ ]:
view = py3Dmol.view(viewergrid=(1,3), width=1000)

# load the original structure file in the left-most cell
view.removeAllModels(viewer=(0,0))
view.addModel(before_model,viewer=(0,0))

# load the temp structure file in the middle
view.removeAllModels(viewer=(0,1))
view.addModel(temp_model,viewer=(0,1))

# load the final modified file in the right-most cell
view.removeAllModels(viewer=(0,2))
view.addModel(after_model,viewer=(0,2))

for i in range(3):
    view.setStyle({'cartoon': {'color': '#ADD8E6'}},viewer=(0,i))
    view.setStyle({'resi':[nsaa_residue_number]}, 
                  {'stick':{'colorscheme':'cyanCarbon'},
                   'cartoon':{'color': 'red'}},
                  viewer=(0,i))

view.render()